In [1]:
import pandas as pd
import os

In [2]:
# Генерация пути к файлу
def filepath(file):
    return os.path.join(os.getcwd(), 'riiid-test-answer-prediction', file)

train.csv: Основной датасет

In [3]:
# Чтение данных кусками (chunk) по 10 миллионов строк за раз, приведение к типу данных, указанных в описании kaggle.
# https://www.kaggle.com/competitions/riiid-test-answer-prediction/data
chunk = pd.read_csv(filepath('train.csv'),
                    chunksize=10_000_000,
                    dtype={
                        'row_id': 'int64',
                        'timestamp': 'int64',
                        'user_id': 'int32',
                        'content_id': 'int16',
                        'content_type_id': 'int8',
                        'task_container_id': 'int16',
                        'user_answer': 'int8',
                        'answered_correctly':'int8',
                        'prior_question_elapsed_time': 'float32',
                        'prior_question_had_explanation': 'boolean'
                        }
                   )
dataset = pd.concat(chunk, ignore_index=True)

In [4]:
dataset.head() # Первые пять строк датафрейма

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:
dataset.tail() # Последние пять строк датафрейма

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
101230327,101230327,428564420,2147482888,3586,0,22,0,1,18000.0,True
101230328,101230328,428585000,2147482888,6341,0,23,3,1,14000.0,True
101230329,101230329,428613475,2147482888,4212,0,24,3,1,14000.0,True
101230330,101230330,428649406,2147482888,6343,0,25,1,0,22000.0,True
101230331,101230331,428692118,2147482888,7995,0,26,3,1,29000.0,True


In [6]:
dataset.shape # 101_230_332 строк, 10 колонок

(101230332, 10)

In [7]:
dataset.info() # Информация о наименовании колонок, типах данных, используемой памяти, кол-ве строк и колонок.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int32  
 3   content_id                      int16  
 4   content_type_id                 int8   
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(2), int8(3)
memory usage: 3.1 GB


In [8]:
dataset.drop(['row_id'], axis=1, inplace=True) # Удаляем дублирующую индексы колонку row_id
dataset.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False


In [9]:
dataset.info() # Уменьшился размер используемой памяти

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 2.4 GB


In [10]:
dataset[['timestamp', 'user_id', 'content_id', 'content_type_id', 'task_container_id', 'user_answer', 'answered_correctly']]\
.isnull().sum() # В первых 7 столбцах отсутствуют пропущенные значения

timestamp             0
user_id               0
content_id            0
content_type_id       0
task_container_id     0
user_answer           0
answered_correctly    0
dtype: int64

In [11]:
dataset[['timestamp', 'user_id', 'content_id', 'content_type_id', 'task_container_id', 'user_answer', 'answered_correctly']]\
.isna().sum() # В первых 7 столбцах отсутствуют пропущенные значения

timestamp             0
user_id               0
content_id            0
content_type_id       0
task_container_id     0
user_answer           0
answered_correctly    0
dtype: int64

In [12]:
users_unique = dataset['user_id'].nunique() # Количество протестированных учащихся - 393_656
users_unique

393656

In [13]:
dataset['user_answer'].value_counts(True) # Ответы учащихся на тесты, доля (4 варианта: 0-А, 1-В, 2-С, 3-D; -1-лекция)

 0    0.278439
 1    0.266620
 3    0.257678
 2    0.177911
-1    0.019352
Name: user_answer, dtype: float64

In [14]:
dataset['content_type_id'].value_counts() # 0-вопросы, 1-лекции

0    99271300
1     1959032
Name: content_type_id, dtype: int64

In [15]:
dataset['content_type_id'].value_counts(True) # 0-вопросы, 1-лекции, доля

0    0.980648
1    0.019352
Name: content_type_id, dtype: float64

In [16]:
# Доля (в %) правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся.
# Группировка идет по учащимся в разрезе вопросов.
answered_correct = dataset[dataset['content_type_id'] == 0].groupby(['user_id'])['answered_correctly']\
.aggregate(lambda x: round(sum(x)/x.count()*100, 2))

In [17]:
answered_correct = answered_correct.reset_index() # Сброс индекса
answered_correct

,user_id,answered_correctly
0,115,69.57
1,124,23.33
2,2746,57.89
3,5382,67.20
4,8623,64.22
...,...,...
393651,2147470770,76.55
393652,2147470777,69.15
393653,2147481750,76.00
393654,2147482216,64.36


In [18]:
# Среднее по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, в % 
round(answered_correct['answered_correctly'].mean(), 2)

54.52

In [19]:
# Медиана по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, в % 
answered_correct['answered_correctly'].median()

57.14

In [20]:
# Кол-во уникальных долей 6363
answered_correct['answered_correctly'].nunique()

6363

In [21]:
# Поэтому разделим на бины, 5 категорий долей (0-20, 20-40, 40-60, 60-80, 80-100)
bins_cat = pd.cut(answered_correct['answered_correctly'], [0, 20, 40, 60, 80, 100])
bins_cat.value_counts(True)
# Примерно 40% каждый имеют бины с долей правильных ответов 40-60, 60-80. Примерно 18% приходится на бины с долей 20-40.
# Около 3% по каждому приходится на бины с долями 0-20, 80-100. 

(60, 80]     0.389326
(40, 60]     0.372305
(20, 40]     0.179025
(0, 20]      0.030228
(80, 100]    0.029115
Name: answered_correctly, dtype: float64

In [22]:
# Посмотрим на успеваемость учащихся, которые слушали лекции (группировка по учащимся в разрезе лекций).
users_lec = dataset[dataset['content_type_id'] == 1]['user_id']
users_lec = set(users_lec)
# Доля в % учащихся, которые слушали лекции во время обучения (38%).
fraction_lec = round(len(users_lec)/users_unique*100, 2)
fraction_lec

38.0

In [23]:
len(users_lec) # Кол-во учащихся, которые слушали лекции (149_606).

149606

In [24]:
# Фрейм, где содержатся вопросы учащимся, которые слушали лекции.
data_lec = dataset[dataset['content_type_id'] == 0]
users_lec_data = data_lec[data_lec['user_id'].isin(users_lec)]
users_lec_data

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
76,0,2746,5273,0,0,1,0,NaN,<NA>
77,21592,2746,758,0,1,0,0,28000.0,False
78,49069,2746,5976,0,2,3,0,17000.0,False
79,72254,2746,236,0,3,3,1,24000.0,False
80,91945,2746,404,0,4,3,0,20000.0,False
...,...,...,...,...,...,...,...,...,...
101230300,1470504008,2147482216,1115,0,275,1,1,16000.0,True
101230301,1470528731,2147482216,1220,0,276,0,1,15000.0,True
101230302,1470551804,2147482216,869,0,277,0,1,18000.0,True
101230303,1470571968,2147482216,1177,0,278,3,1,16000.0,True


In [25]:
users_lec_data['user_id'].nunique() # Проверка на кол-во уникальных учащихся, которые слушали лекции (149_606).

149606

In [26]:
# Доля (в %) правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся, который слушал лекции.
# Группировка идет по учащимся в разрезе вопросов.
answered_correctly_lec = users_lec_data.groupby(['user_id'])['answered_correctly']\
.aggregate(lambda x: round(sum(x)/x.count()*100, 2))

In [27]:
answered_correctly_lec = answered_correctly_lec.reset_index() # Сброс индекса
answered_correctly_lec

,user_id,answered_correctly
0,2746,57.89
1,5382,67.20
2,8623,64.22
3,12741,57.36
4,13134,70.64
...,...,...
149601,2147419988,50.77
149602,2147469944,73.26
149603,2147470770,76.55
149604,2147470777,69.15


In [28]:
# Среднее по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который слушал лекции, в % 
round(answered_correctly_lec['answered_correctly'].mean(), 2)

61.29

In [29]:
# Медиана по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который слушал лекции, в % 
answered_correctly_lec['answered_correctly'].median()

63.02

In [30]:
# Максимум по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который слушал лекции, в % 
answered_correctly_lec['answered_correctly'].max()

100.0

In [31]:
# Минимум по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который слушал лекции, в % 
answered_correctly_lec['answered_correctly'].min()

4.35

In [32]:
# Кол-во уникальных долей 6072
answered_correctly_lec['answered_correctly'].nunique()

6072

In [33]:
# Поэтому разделим на бины, 5 категорий долей
bins_cat_lec = pd.cut(answered_correctly_lec['answered_correctly'], [0, 20, 40, 60, 80, 100])
bins_cat_lec.value_counts(True)
# Около 58% имеет бин с долей правильных ответов 60-80. Примерно 34% приходится на бин с долей 40-60.
# Около 6% приходится на бин с долями 20-40. Около 2,5% имеет бин с долей правильных ответов 80-100.
# И менее 0,2% приходится на бин с долями 0-20.

(60, 80]     0.577778
(40, 60]     0.338182
(20, 40]     0.057224
(80, 100]    0.025079
(0, 20]      0.001738
Name: answered_correctly, dtype: float64

In [34]:
# Посмотрим на успеваемость учащихся, которые не слушали лекции (группировка по учащимся в разрезе лекций).
users_all = set(dataset['user_id'])
users_wo_lec = users_all - users_lec
# Доля в % учащихся, которые не слушали лекции во время обучения (62%).
fraction_wo_lec = round(len(users_wo_lec)/users_unique*100, 2)
fraction_wo_lec

62.0

In [35]:
len(users_wo_lec) # Кол-во учащихся, которые не слушали лекции (244_050).

244050

In [36]:
# Фрейм, где содержатся вопросы учащимся, которые не слушали лекции.
users_wo_lec_data = data_lec[data_lec['user_id'].isin(users_wo_lec)]
users_wo_lec_data

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False
...,...,...,...,...,...,...,...,...,...
101230327,428564420,2147482888,3586,0,22,0,1,18000.0,True
101230328,428585000,2147482888,6341,0,23,3,1,14000.0,True
101230329,428613475,2147482888,4212,0,24,3,1,14000.0,True
101230330,428649406,2147482888,6343,0,25,1,0,22000.0,True


In [37]:
users_wo_lec_data['user_id'].nunique() # Проверка на кол-во уникальных учащихся, которые не слушали лекции (244_050).

244050

In [38]:
# Доля (в %) правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся, который не слушал лекции.
# Группировка идет по учащимся в разрезе вопросов.
answered_correctly_wo_lec = users_wo_lec_data.groupby(['user_id'])['answered_correctly']\
.aggregate(lambda x: round(sum(x)/x.count()*100, 2))

In [39]:
answered_correctly_wo_lec = answered_correctly_wo_lec.reset_index() # Сброс индекса
answered_correctly_wo_lec

,user_id,answered_correctly
0,115,69.57
1,124,23.33
2,8701,58.82
3,24600,34.00
4,32421,46.67
...,...,...
244045,2147457494,79.49
244046,2147463192,58.82
244047,2147464207,42.00
244048,2147481750,76.00


In [40]:
# Среднее по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который не слушал лекции, в % 
# Средняя по доли тех, кто слушал лекции - 61.29
round(answered_correctly_wo_lec['answered_correctly'].mean(), 2)

50.37

In [41]:
# Медиана по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который не слушал лекции, в % 
answered_correctly_wo_lec['answered_correctly'].median()

51.43

In [42]:
# Максимум по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который не слушал лекции, в % 
answered_correctly_wo_lec['answered_correctly'].max()

100.0

In [43]:
# Минимум по доли правильных ответов к общему кол-ву вопросов, заданных к каждому учащемуся, который не слушал лекции, в % 
answered_correctly_wo_lec['answered_correctly'].min()

0.0

In [44]:
# Кол-во уникальных долей 4972
answered_correctly_wo_lec['answered_correctly'].nunique()

4972

In [45]:
# Поэтому разделим на бины, 5 категорий долей
bins_cat_wo_lec = pd.cut(answered_correctly_wo_lec['answered_correctly'], [0, 20, 40, 60, 80, 100])
bins_cat_wo_lec.value_counts(True)
# Около 39% имеет бин с долей правильных ответов 40-60 (34% тех, кто слушал лекции).
# Примерно 27% приходится на бин с долей 60-80 (58% тех, кто слушал лекции).
# Около 25% приходится на бин с долями 20-40 (6% тех, кто слушал лекции).
# Около 5% имеет бин с долей правильных ответов 0-20 (менее 0,2% тех, кто слушал лекции).
# И примерно 3% приходится на бин с долями 80-100 (2,5% тех, кто слушал лекции).

(40, 60]     0.393240
(60, 80]     0.273714
(20, 40]     0.253748
(0, 20]      0.047707
(80, 100]    0.031591
Name: answered_correctly, dtype: float64

In [46]:
dataset['task_container_id'].value_counts() # 10_000 уникальных значений колонки task_container_id

14      804285
15      798539
4       692079
5       690051
6       688813
         ...  
9932       174
9999       174
9937       172
9925       172
9926       170
Name: task_container_id, Length: 10000, dtype: int64

In [47]:
# Рассмотрим колонку timestamp - время в миллисекундах между этим взаимодействием с пользователем и первым завершением 
# события от этого пользователя.
# Группировка по учащимся в разрезе общего кол-ва времени в миллисекундах, потраченное на обучение 
# (прослушивание лекций и/или ответы на вопросы) и общее кол-во действий (ответ на вопрос или лекция) для учащегося.
agg_func = {'timestamp': ['last'], 'user_id': ['count']}
timestamp_data = dataset.groupby(['user_id']).aggregate(agg_func)
timestamp_data

,timestamp,user_id
,last,count
user_id,,
115,668090043,46
124,571323,30
2746,835457,20
5382,2101551456,128
8623,862338736,112
...,...,...
2147470770,2832089444,228
2147470777,13332685203,758


In [48]:
timestamp_data.reset_index(inplace=True) # Сброс индекса

In [49]:
# Добавляем колонку среднее время на одно действие в миллисекундах.
timestamp_data[('avg_time', 'milliseconds per action')] = \
round(timestamp_data[('timestamp', 'last')]/timestamp_data[('user_id', 'count')], 2)
timestamp_data

,user_id,timestamp,user_id,avg_time
,,last,count,milliseconds per action
0,115,668090043,46,14523696.59
1,124,571323,30,19044.10
2,2746,835457,20,41772.85
3,5382,2101551456,128,16418370.75
4,8623,862338736,112,7699453.00
...,...,...,...,...
393651,2147470770,2832089444,228,12421444.93
393652,2147470777,13332685203,758,17589294.46
393653,2147481750,55954768,50,1119095.36


In [50]:
# Объединение timestamp_data с группированной таблицей по учащимся и доли в % правильных ответов к общему кол-ву вопросов,
# заданных каждому учащемуся.
merge_timestamp_data = pd.merge(timestamp_data, answered_correct, how='left', left_on=[('user_id', '')], right_on=['user_id'])
merge_timestamp_data

F:\Programs\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(user_id, )","(timestamp, last)","(user_id, count)","(avg_time, milliseconds per action)",user_id,answered_correctly
0,115,668090043,46,14523696.59,115,69.57
1,124,571323,30,19044.10,124,23.33
2,2746,835457,20,41772.85,2746,57.89
3,5382,2101551456,128,16418370.75,5382,67.20
4,8623,862338736,112,7699453.00,8623,64.22
...,...,...,...,...,...,...
393651,2147470770,2832089444,228,12421444.93,2147470770,76.55
393652,2147470777,13332685203,758,17589294.46,2147470777,69.15
393653,2147481750,55954768,50,1119095.36,2147481750,76.00
393654,2147482216,1470594073,280,5252121.69,2147482216,64.36


In [51]:
merge_timestamp_data.drop(['user_id'], axis=1, inplace=True) # Удаление дублирующей колонки user_id
merge_timestamp_data

,"(user_id, )","(timestamp, last)","(user_id, count)","(avg_time, milliseconds per action)",answered_correctly
0,115,668090043,46,14523696.59,69.57
1,124,571323,30,19044.10,23.33
2,2746,835457,20,41772.85,57.89
3,5382,2101551456,128,16418370.75,67.20
4,8623,862338736,112,7699453.00,64.22
...,...,...,...,...,...
393651,2147470770,2832089444,228,12421444.93,76.55
393652,2147470777,13332685203,758,17589294.46,69.15
393653,2147481750,55954768,50,1119095.36,76.00
393654,2147482216,1470594073,280,5252121.69,64.36


In [52]:
# Переименование колонок фрейма merge_timestamp_data
merge_timestamp_data.set_axis\
(['user_id', 'timestamp, last', 'user_id, count', 'avg_time, milliseconds per action', 'answered_correctly'], \
 axis='columns', inplace=True)
merge_timestamp_data

,user_id,"timestamp, last","user_id, count","avg_time, milliseconds per action",answered_correctly
0,115,668090043,46,14523696.59,69.57
1,124,571323,30,19044.10,23.33
2,2746,835457,20,41772.85,57.89
3,5382,2101551456,128,16418370.75,67.20
4,8623,862338736,112,7699453.00,64.22
...,...,...,...,...,...
393651,2147470770,2832089444,228,12421444.93,76.55
393652,2147470777,13332685203,758,17589294.46,69.15
393653,2147481750,55954768,50,1119095.36,76.00
393654,2147482216,1470594073,280,5252121.69,64.36


In [53]:
# Вывод описательной статистики для колонки среднее время на одно действие в миллисекундах.
merge_timestamp_data['avg_time, milliseconds per action'].describe()

count    3.936560e+05
mean     4.811595e+07
std      1.718847e+08
min      0.000000e+00
25%      4.161821e+04
50%      3.901691e+06
75%      2.368890e+07
max      9.864038e+09
Name: avg_time, milliseconds per action, dtype: float64

In [54]:
# Добавление новой колонки cat_score к merge_timestamp_data - категории долей правильных ответов.
def get_cat_score(arg):
    if arg >= 0 and arg <=20:
        return '0-20'
    elif arg > 20 and arg <= 40:
        return '20-40'
    elif arg > 40 and arg <= 60:
        return '40-60'
    elif arg > 60 and arg <= 80:
        return '60-80'
    elif arg > 80 and arg <=100:
        return '80-100'
merge_timestamp_data['cat_score'] = merge_timestamp_data['answered_correctly'].apply(get_cat_score)
merge_timestamp_data['cat_score']

0         60-80
1         20-40
2         40-60
3         60-80
4         60-80
          ...  
393651    60-80
393652    60-80
393653    60-80
393654    60-80
393655    40-60
Name: cat_score, Length: 393656, dtype: object

In [55]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и среднему времени действия во время обучения.
# Выведение описательной статистики по среднему времени в разрезе категорий долей правильных ответов.
agg_stat = {'avg_time, milliseconds per action': ['describe']}
timestamp_score_data = merge_timestamp_data.groupby(['cat_score']).aggregate(agg_stat)
timestamp_score_data
# Большая часть учащихся получала от 40% до 80% правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся.
# В среднем, среднее время одного действия учащегося в разрезе категорий долей правильных ответов во время обучения 
# 3-6*10^7 миллисекунд
# По минимальному значению видно, что в пределах 0-40%, 80-100% timestamp равен 0 (время обучения = 0). Ниже показано
# что 89 учащихся имеют timestamp=0.
# По медиане видно, что среднее время действия обучения больше у учащихся с 60-80%.
# Максимум приходится на категорию от 0-20%. При детальном рассмотрении (ниже указано) выяснено, что такое время имеет
# user с id = 916468081, ответил неправильно на два вопроса и долгое ожидание ответа на второй вопрос.
# Далее максимум имеют категории 20-40%, 40-60%.

avg_time, milliseconds per action                              \
                                   describe                               
                                      count          mean           std   
cat_score                                                                 
0-20                                12080.0  3.320922e+07  2.320257e+08   
20-40                               70441.0  5.000840e+07  2.104943e+08   
40-60                              146491.0  5.711391e+07  1.853526e+08   
60-80                              153188.0  4.107208e+07  1.288373e+08   
80-100                              11456.0  3.132859e+07  1.500235e+08   

                                                                           
                                                                           
               min           25%          50%           75%           max  
cat_score                                                                  
0-20          0.00  1.751235e+04    27186.575  5.027986e+04  9.864038e+09  
20-40         0.00  2.750858e+04    41644.340  5.657507e+06  7.519464e+09  
40-60      2034.87  4.446636e+04  4067115.890  3.001058e+07  7.452502e+09  
60-80      6195.67  1.726948e+06  7775459.950  2.821457e+07  5.455826e+09  
80-100        0.00  3.615499e+04  2087267.320  9.683945e+06  4.038946e+09

In [56]:
merge_timestamp_data[(merge_timestamp_data['timestamp, last'] == 0) & (merge_timestamp_data['user_id, count'] == 1)]
# 87 учащихся имеют timestamp - 0 (время обучения = 0) и ответили на один вопрос

,user_id,"timestamp, last","user_id, count","avg_time, milliseconds per action",answered_correctly,cat_score
308,1880240,0,1,0.0,100.0,80-100
3661,19887248,0,1,0.0,0.0,0-20
7007,38532797,0,1,0.0,0.0,0-20
8806,48251651,0,1,0.0,100.0,80-100
11057,60215278,0,1,0.0,100.0,80-100
...,...,...,...,...,...,...
370109,2019867894,0,1,0.0,0.0,0-20
376271,2053688599,0,1,0.0,0.0,0-20
381370,2081720057,0,1,0.0,0.0,0-20
385372,2103184240,0,1,0.0,100.0,80-100


In [57]:
merge_timestamp_data[(merge_timestamp_data['timestamp, last'] == 0) & (merge_timestamp_data['user_id, count'] > 1)]
# 2 учащихся имеют timestamp - 0 (время обучения = 0) и ответили на 3-4 вопроса

,user_id,"timestamp, last","user_id, count","avg_time, milliseconds per action",answered_correctly,cat_score
31331,171953341,0,3,0.0,33.33,20-40
358879,1958652827,0,4,0.0,25.00,20-40


In [58]:
# Выбор среднего времени более, чем 9e+9 миллисекунд
merge_timestamp_data[merge_timestamp_data['avg_time, milliseconds per action'] > 9e+9]

,user_id,"timestamp, last","user_id, count","avg_time, milliseconds per action",answered_correctly,cat_score
167295,916468081,19728076593,2,9.864038e+09,0.0,0-20


In [59]:
# Просмотр данных об учащемся с id=916468081
dataset[dataset['user_id'] == 916468081]

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
43253552,0,916468081,4136,0,0,3,0,NaN,<NA>
43253553,19728076593,916468081,3878,0,1,0,0,19000.0,True


In [60]:
# Рассмотрим колонку prior_question_elapsed_time - среднее время в миллисекундах, затрачиваемое пользователем на ответ на 
# каждый вопрос из предыдущей группы вопросов, без учета промежуточных лекций. Имеет значение null для первого набора 
# вопросов пользователя или лекции.
# Группировка по учащимся в разрезе описательной статистики среднего времени в миллисекундах, потраченное на ответ.
agg_stat_avg = {'prior_question_elapsed_time': ['max', 'min', 'median', 'mean']}
avg_time_data = dataset.groupby(['user_id']).aggregate(agg_stat_avg)
avg_time_data

prior_question_elapsed_time                                
                                   max      min   median          mean
user_id                                                               
115                            55000.0   5000.0  20000.0  19933.310547
124                            33333.0   6500.0  21000.0  18793.000000
2746                           28000.0  13000.0  17500.0  18055.554688
5382                          201000.0   5000.0  25000.0  36048.386719
8623                           95000.0   5000.0  20000.0  26107.408203
...                                ...      ...      ...           ...
2147470770                    123000.0   3000.0  19000.0  23167.382812
2147470777                     91000.0   2000.0  20000.0  20028.103516
2147481750                     33000.0   3000.0  17000.0  17938.775391
2147482216                     59000.0   6000.0  18000.0  19562.042969
2147482888                    102000.0   9000.0  23500.0  27192.308594

[393656 rows x 4 columns]

In [61]:
avg_time_data.reset_index(inplace=True) # Сброс индекса

In [62]:
# Объединение avg_time_data с группированной таблицей по учащимся и доли в % правильных ответов к общему кол-ву вопросов,
# заданных каждому учащемуся.
merge_avg_time_data = pd.merge(avg_time_data, answered_correct, how='left', left_on=[('user_id', '')], right_on=['user_id'])
merge_avg_time_data

F:\Programs\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(user_id, )","(prior_question_elapsed_time, max)","(prior_question_elapsed_time, min)","(prior_question_elapsed_time, median)","(prior_question_elapsed_time, mean)",user_id,answered_correctly
0,115,55000.0,5000.0,20000.0,19933.310547,115,69.57
1,124,33333.0,6500.0,21000.0,18793.000000,124,23.33
2,2746,28000.0,13000.0,17500.0,18055.554688,2746,57.89
3,5382,201000.0,5000.0,25000.0,36048.386719,5382,67.20
4,8623,95000.0,5000.0,20000.0,26107.408203,8623,64.22
...,...,...,...,...,...,...,...
393651,2147470770,123000.0,3000.0,19000.0,23167.382812,2147470770,76.55
393652,2147470777,91000.0,2000.0,20000.0,20028.103516,2147470777,69.15
393653,2147481750,33000.0,3000.0,17000.0,17938.775391,2147481750,76.00
393654,2147482216,59000.0,6000.0,18000.0,19562.042969,2147482216,64.36


In [63]:
merge_avg_time_data.drop(['user_id'], axis=1, inplace=True) # Удаление дублирующей колонки user_id

In [64]:
# Переименование колонок фрейма merge_avg_time_data
merge_avg_time_data.set_axis\
(['user_id', 'prior_question_elapsed_time, max', 'prior_question_elapsed_time, min', 'prior_question_elapsed_time, median',\
  'prior_question_elapsed_time, mean', 'answered_correctly'], axis='columns', inplace=True)
merge_avg_time_data

,user_id,"prior_question_elapsed_time, max","prior_question_elapsed_time, min","prior_question_elapsed_time, median","prior_question_elapsed_time, mean",answered_correctly
0,115,55000.0,5000.0,20000.0,19933.310547,69.57
1,124,33333.0,6500.0,21000.0,18793.000000,23.33
2,2746,28000.0,13000.0,17500.0,18055.554688,57.89
3,5382,201000.0,5000.0,25000.0,36048.386719,67.20
4,8623,95000.0,5000.0,20000.0,26107.408203,64.22
...,...,...,...,...,...,...
393651,2147470770,123000.0,3000.0,19000.0,23167.382812,76.55
393652,2147470777,91000.0,2000.0,20000.0,20028.103516,69.15
393653,2147481750,33000.0,3000.0,17000.0,17938.775391,76.00
393654,2147482216,59000.0,6000.0,18000.0,19562.042969,64.36


In [65]:
# Вывод описательной статистики для фрейма merge_avg_time_data.
merge_avg_time_data.describe()

,user_id,"prior_question_elapsed_time, max","prior_question_elapsed_time, min","prior_question_elapsed_time, median","prior_question_elapsed_time, mean",answered_correctly
count,3.936560e+05,393569.00000,393569.000000,393569.000000,393569.000000,393656.000000
mean,1.076358e+09,82630.31250,6656.581055,21202.005859,23894.033203,54.518335
std,6.201319e+08,69458.09375,4864.332031,6694.925293,7965.080078,16.337853
min,1.150000e+02,0.00000,0.000000,0.000000,0.000000,0.000000
25%,5.387596e+08,40000.00000,3000.000000,17500.000000,19114.583984,43.330000
50%,1.077717e+09,57000.00000,6000.000000,20000.000000,22722.650391,57.140000
75%,1.613533e+09,92000.00000,9000.000000,24000.000000,27308.992188,66.670000
max,2.147483e+09,300000.00000,91000.000000,300000.000000,230833.328125,100.000000


In [66]:
# Добавление новой колонки cat_score к merge_avg_time_data - категории долей правильных ответов.
merge_avg_time_data['cat_score'] = merge_avg_time_data['answered_correctly'].apply(get_cat_score)
merge_avg_time_data['cat_score']

0         60-80
1         20-40
2         40-60
3         60-80
4         60-80
          ...  
393651    60-80
393652    60-80
393653    60-80
393654    60-80
393655    40-60
Name: cat_score, Length: 393656, dtype: object

In [67]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и среднему из среднего времени на ответ.
# Выведение описательной статистики по среднему времени в разрезе категорий долей правильных ответов.
agg_stat_avg_time = {'prior_question_elapsed_time, mean': ['describe']}
avg_time_score_data = merge_avg_time_data.groupby(['cat_score']).aggregate(agg_stat_avg_time)
avg_time_score_data
# Большая часть учащихся получала от 40% до 80% правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся.
# В среднем, среднее время ответа учащегося в разрезе категорий долей правильных ответов во время обучения 
# 19,7-25*10^3 миллисекунд
# Минимальное значение равно 0 по всем категориям.
# По медиане видно, что среднее время ответа немного превышает остальные у учащихся с 40-60%.
# Максимум приходится на категорию от 40-60%. Далее максимум имеет категория 60-80%.

prior_question_elapsed_time, mean                                  \
                                   describe                                   
                                      count          mean          std  min   
cat_score                                                                     
0-20                                12033.0  19730.242188  9999.092773  0.0   
20-40                               70441.0  23514.476562  9335.868164  0.0   
40-60                              146491.0  24953.201172  8174.017090  0.0   
60-80                              153188.0  23549.753906  6697.477539  0.0   
80-100                              11416.0  21638.925781  6694.336426  0.0   

                                                                    
                                                                    
                    25%           50%           75%            max  
cat_score                                                           
0-20       13275.724609  19643.517578  25160.724609  143600.000000  
20-40      18331.363281  22727.273438  27770.000000  137909.093750  
40-60      19812.500000  23437.500000  28252.141602  230833.328125  
60-80      19150.000000  22379.258789  26519.559570  224300.000000  
80-100     17444.242188  20610.589844  24670.023438  157363.640625

In [68]:
# Рассмотрим колонку prior_question_had_explanation - видел ли пользователь объяснение и правильный ответ после ответа 
# на предыдущий пакет вопросов, игнорируя любые промежуточные лекции. Это значение является общим для одного пакета вопросов 
# и равно нулю для первого пакета вопросов или лекции пользователя.
dataset['prior_question_had_explanation'].value_counts() # Абсолютное кол-во объяснений и отсутствие объяснений (после ответа,
# после первых ответов, лекции).

True     89685560
False    11152266
Name: prior_question_had_explanation, dtype: Int64

In [69]:
# Доля объяснений и отсутствие объяснений (после ответа, после первых ответов, лекции).
dataset['prior_question_had_explanation'].value_counts(True)

True     0.889404
False    0.110596
Name: prior_question_had_explanation, dtype: float64

In [70]:
dataset['prior_question_had_explanation'].isnull().sum() # Кол-во пустых значений 392_506

392506

In [71]:
dataset['prior_question_had_explanation'].unique() # Уникальные значения в колонке prior_question_had_explanation

<BooleanArray>
[<NA>, False, True]
Length: 3, dtype: boolean

In [72]:
# Группировка по учащимся в разрезе доли было объяснение или нет.
explain_data = dataset.groupby(['user_id'])['prior_question_had_explanation'].value_counts(True)
explain_data = explain_data.unstack() # Перевод true, false из индексов в колонки.

In [73]:
explain_data.reset_index(inplace=True) # Сброс индекса

In [74]:
# Объединение explain_data с группированной таблицей по учащимся и доли в % правильных ответов к общему кол-ву вопросов,
# заданных каждому учащемуся.
merge_explain_data = pd.merge(explain_data, answered_correct, how='left', on=['user_id'])
merge_explain_data

,user_id,False,True,answered_correctly
0,115,0.866667,0.133333,69.57
1,124,1.000000,NaN,23.33
2,2746,0.421053,0.578947,57.89
3,5382,0.110236,0.889764,67.20
4,8623,0.135135,0.864865,64.22
...,...,...,...,...
393564,2147470770,0.140969,0.859031,76.55
393565,2147470777,0.047556,0.952444,69.15
393566,2147481750,0.204082,0.795918,76.00
393567,2147482216,0.046595,0.953405,64.36


In [75]:
# Вывод описательной статистики для фрейма merge_avg_time_data.
merge_explain_data.describe()

,user_id,False,True,answered_correctly
count,3.935690e+05,392062.000000,317009.000000,393569.000000
mean,1.076369e+09,0.443631,0.692845,54.520223
std,6.201310e+08,0.352243,0.246986,16.322354
min,1.150000e+02,0.000287,0.004673,0.000000
25%,5.387605e+08,0.115044,0.529412,43.330000
50%,1.077746e+09,0.357143,0.750000,57.140000
75%,1.613547e+09,0.769231,0.911392,66.670000
max,2.147483e+09,1.000000,1.000000,100.000000


In [76]:
merge_explain_data.columns # Наименования колонок merge_explain_data

Index(['user_id', False, True, 'answered_correctly'], dtype='object')

In [77]:
# Добавление новой колонки cat_score к merge_explain_data - категории долей правильных ответов.
merge_explain_data['cat_score'] = merge_explain_data['answered_correctly'].apply(get_cat_score)
merge_explain_data['cat_score']

0         60-80
1         20-40
2         40-60
3         60-80
4         60-80
          ...  
393564    60-80
393565    60-80
393566    60-80
393567    60-80
393568    40-60
Name: cat_score, Length: 393569, dtype: object

In [78]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и присутствию/отсутствию объяснения на вопрос.
# Выведение описательной статистики по присутствию/отсутствию объяснения на вопрос в разрезе категорий долей правильных ответов.
agg_stat_explain = {True: ['describe'], False: ['describe']}
explain_score_data = merge_explain_data.groupby(['cat_score']).aggregate(agg_stat_explain)
explain_score_data
# Большая часть учащихся получала от 40% до 80% правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся.
# По среднему и медиане, объяснение ответа на вопрос немного больше в категории 60-80%, далее идет 80-100%, 40-60%. 
# По среднему и медиане отсутствие объяснения больше в категориях 0-20, 20-40%.
# Минимальное значение при объяснении ответа на вопрос колеблется примерно от 0,5-3% по всем категориям, при отсутствии 
# объяснения 1% и менее.
# Максимум по всем категориям равно 1 (100%).

True                                                     \
           describe                                                     
              count      mean       std       min       25%       50%   
cat_score                                                               
0-20         2866.0  0.344215  0.204143  0.031250  0.166667  0.333333   
20-40       35948.0  0.469708  0.242211  0.012346  0.250000  0.473684   
40-60      124329.0  0.645494  0.238036  0.009302  0.473684  0.677419   
60-80      144471.0  0.791506  0.199109  0.004673  0.703704  0.874046   
80-100       9395.0  0.762451  0.223480  0.008264  0.633333  0.844444   

                             False                                          \
                          describe                                           
                75%  max     count      mean       std       min       25%   
cat_score                                                                    
0-20       0.523810  1.0   12024.0  0.918703  0.175545  0.011905  1.000000   
20-40      0.638889  1.0   70324.0  0.761560  0.290259  0.002165  0.526316   
40-60      0.859155  1.0  146069.0  0.453467  0.318260  0.001320  0.171053   
60-80      0.939394  1.0  152419.0  0.254813  0.266536  0.000287  0.063910   
80-100     0.946032  1.0   11226.0  0.378832  0.354322  0.001014  0.072760   

                                    
                                    
                50%       75%  max  
cat_score                           
0-20       1.000000  1.000000  1.0  
20-40      0.966667  1.000000  1.0  
40-60      0.413793  0.687500  1.0  
60-80      0.141414  0.363636  1.0  
80-100     0.269231  0.588235  1.0

lectures.csv: метаданные для лекций, просмотренных пользователями по мере их обучения.

In [79]:
lectures_data = pd.read_csv(filepath('lectures.csv'), sep=',') # Данные по лекциям
lectures_data

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question
...,...,...,...,...
413,32535,8,5,solving question
414,32570,113,3,solving question
415,32604,24,6,concept
416,32625,142,2,concept


In [80]:
lectures_data['lecture_id'].nunique() # 418 лекций

418

In [81]:
lectures_data.info() # 418 строк и лекций соответственно, 4 столбца без пропущенных значений.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


In [82]:
lectures_data['tag'].nunique() # 151 уникальный tag

151

In [83]:
lectures_data[lectures_data['tag'] == 0] # Есть tag равные 0

,lecture_id,tag,part,type_of
118,10052,0,7,concept
407,32146,0,7,solving question


In [84]:
lectures_data['part'].value_counts() # Части тестирования с 1 по 7

5    143
6     83
2     56
1     54
7     32
4     31
3     19
Name: part, dtype: int64

In [85]:
lectures_data['type_of'].value_counts() # Типы основной цели лекций.

concept             222
solving question    186
intention             7
starter               3
Name: type_of, dtype: int64

In [86]:
# Добавление нового столбца section.
# Секция Listening (part 1, 2, 3, 4) (дается 45 минут, 100 вопросов).
# Секция Reading (part 5, 6, 7) (дается 75 минут, 100 вопросов).
listening = range(1, 5)
lectures_data['section'] = lectures_data['part'].apply(lambda x: 'listening' if x in listening else 'reading')
lectures_data

,lecture_id,tag,part,type_of,section
0,89,159,5,concept,reading
1,100,70,1,concept,listening
2,185,45,6,concept,reading
3,192,79,5,solving question,reading
4,317,156,5,solving question,reading
...,...,...,...,...,...
413,32535,8,5,solving question,reading
414,32570,113,3,solving question,listening
415,32604,24,6,concept,reading
416,32625,142,2,concept,listening


In [87]:
lectures_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
 4   section     418 non-null    object
dtypes: int64(3), object(2)
memory usage: 16.5+ KB


In [88]:
lectures_data['section'].value_counts(True) 
# Доля лекций секции Reading примерно 62%, она соотносится с долей минут данной секции на экзамене - 75 минут из 120 минут.
# Аналогично для секции Listening.

reading      0.617225
listening    0.382775
Name: section, dtype: float64

In [89]:
set(lectures_data['lecture_id'].values) - set(dataset[dataset['content_type_id'] == 1]['content_id'].values)
# id лекций, которые не были использованы учащимися.

{641, 4385, 28098}

In [90]:
lectures_data[
    (lectures_data['lecture_id'] == 641) | 
    (lectures_data['lecture_id'] == 4385) | 
    (lectures_data['lecture_id'] == 28098)
]
# Не использовались 3 лекции из частей 5 и 6 секции Reading, тип лекций solving question и starter.

,lecture_id,tag,part,type_of,section
7,641,134,6,solving question,reading
54,4385,181,5,starter,reading
355,28098,166,6,solving question,reading


questions.csv: метаданные для вопросов, заданных учащимся.

In [91]:
questions_data = pd.read_csv(filepath('questions.csv'), sep=',') # Данные по вопросам
questions_data

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [92]:
questions_data['question_id'].nunique() # 13_523 вопроса

13523

In [93]:
questions_data.info()
# 13_523 строки и вопросов соответственно, 5 колонок без пропущенных значений,
# за исключением колонки tags (1 пропущенное значение).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


In [94]:
questions_data[questions_data['tags'].isnull()] # Найдено пропущенное значение в колонке tags.

,question_id,bundle_id,correct_answer,part,tags
10033,10033,10033,2,6,NaN


In [95]:
questions_data[questions_data['tags'] == 0] # tags с 0 нет

,question_id,bundle_id,correct_answer,part,tags


In [96]:
questions_data['bundle_id'].nunique() # 9765 уникальных кодов, для которых вопросы подаются вместе.

9765

In [97]:
questions_data[questions_data['question_id'] != questions_data['bundle_id']] # Проверка повторяемости 3758 строк.

,question_id,bundle_id,correct_answer,part,tags
1401,1401,1400,0,3,136 92 102
1402,1402,1400,1,3,82 92 102
1404,1404,1403,0,3,136 38 29
1405,1405,1403,3,3,82 38 29
1407,1407,1406,3,3,136 38 102
...,...,...,...,...,...
13246,13246,13244,1,3,136 81 92
13248,13248,13247,2,3,136 81 92
13249,13249,13247,1,3,136 81 92
13251,13251,13250,2,3,136 81 92


In [98]:
questions_data['correct_answer'].value_counts(True) # 4 варианта ответов в долях: 0-А, 1-В, 2-С, 3-D.

0    0.274791
3    0.262072
1    0.257191
2    0.205945
Name: correct_answer, dtype: float64

In [99]:
questions_data['part'].value_counts() # Части тестирования с 1 по 7

5    5511
2    1647
3    1562
4    1439
6    1212
7    1160
1     992
Name: part, dtype: int64

In [100]:
questions_data['tags'].nunique() # 1519 уникальных tags

1519

In [101]:
# Добавление нового столбца section.
# Секция Listening (part 1, 2, 3, 4) (дается 45 минут, 100 вопросов).
# Секция Reading (part 5, 6, 7) (дается 75 минут, 100 вопросов).
questions_data['section'] = questions_data['part'].apply(lambda x: 'listening' if x in listening else 'reading')
questions_data

,question_id,bundle_id,correct_answer,part,tags,section
0,0,0,0,1,51 131 162 38,listening
1,1,1,1,1,131 36 81,listening
2,2,2,0,1,131 101 162 92,listening
3,3,3,0,1,131 149 162 29,listening
4,4,4,3,1,131 5 162 38,listening
...,...,...,...,...,...,...
13518,13518,13518,3,5,14,reading
13519,13519,13519,3,5,8,reading
13520,13520,13520,2,5,73,reading
13521,13521,13521,0,5,125,reading


In [102]:
questions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
 5   section         13523 non-null  object
dtypes: int64(4), object(2)
memory usage: 634.0+ KB


In [103]:
questions_data['section'].value_counts(True) # Доля вопросов секции Reading преобладает над долей вопросов секции Listening.

reading      0.582933
listening    0.417067
Name: section, dtype: float64

In [104]:
set(questions_data['question_id'].values) - set(dataset[dataset['content_type_id'] == 0]['content_id'].values)
# id вопросов, которые не были заданы учащимся. Все вопросы были заданы.

set()

Объединение трех датафреймов по ключам.
Из dataset (train.csv): content_type_id (1-лекция, 0-вопрос), content_id (id лекций, вопросов).
Из questions_data (questions.csv): question_id (id вопросов). 
Из lectures_data (lectures.csv): lecture_id (id лекций).

In [105]:
questions_data.rename(columns={'part': 'part_q', 'section': 'section_q'}, inplace=True)
# Переименование колонок part и section в questions_data

In [106]:
questions_data['temp_q'] = 0 # Временная колонка для объединения фреймов по двум колонкам (ключам)

In [107]:
lectures_data.rename(columns={'part': 'part_lec', 'section': 'section_lec'}, inplace=True)
# Переименование колонок part и section в lectures_data

In [108]:
lectures_data['temp_lec'] = 1 # Временная колонка для объединения фреймов по двум колонкам (ключам)

In [109]:
merge_dataset = pd.merge(dataset, questions_data, how='left', left_on=['content_type_id', 'content_id'], \
                         right_on=['temp_q', 'question_id'])
# Объединение фреймов по двум ключам - типу контента (вопросы) и id контента

In [110]:
merge_dataset.drop(['question_id', 'temp_q'], axis=1, inplace=True)
# Удаление дублирующей колонки с id вопросов и временной колонки для объединения фреймов
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 14 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
 9   bundle_id                       float64
 10  correct_answer                  float64
 11  part_q                          float64
 12  tags                            object 
 13  section_q                       object 
dtypes: boolean(1), float32(1), float64(3), int16(2), int32(1), int64(1), int8(3), object(2)
memory usage: 6.9+ GB


In [111]:
merge_dataset = pd.merge(merge_dataset, lectures_data, how='left', left_on=['content_type_id', 'content_id'], \
                         right_on=['temp_lec', 'lecture_id'])
# Объединение фреймов по двум ключам - типу контента (лекции) и id контента

In [112]:
merge_dataset.drop(['lecture_id', 'temp_lec'], axis=1, inplace=True)
# Удаление дублирующей колонки с id лекций и временной колонки для объединения фреймов
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
 9   bundle_id                       float64
 10  correct_answer                  float64
 11  part_q                          float64
 12  tags                            object 
 13  section_q                       object 
 14  tag                             float64
 15  part_lec                        float64
 16  type_of                         object 
 17  section_lec            

In [113]:
# Приведение колонок bundle_id и correct_answer к типу данных float 32 и float 16 соответственно, для уменьшения memory usage
merge_dataset['bundle_id'] = merge_dataset['bundle_id'].astype('float32')
merge_dataset['correct_answer'] = merge_dataset['correct_answer'].astype('float16')
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
 9   bundle_id                       float32
 10  correct_answer                  float16
 11  part_q                          float64
 12  tags                            object 
 13  section_q                       object 
 14  tag                             float64
 15  part_lec                        float64
 16  type_of                         object 
 17  section_lec            

In [114]:
merge_dataset[['part_q', 'part_lec']] = merge_dataset[['part_q', 'part_lec']].fillna(0)
# Заполнение пустых значений нулями для последующего объединения колонок part_q и part_lec
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 18 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
 9   bundle_id                       float32
 10  correct_answer                  float16
 11  part_q                          float64
 12  tags                            object 
 13  section_q                       object 
 14  tag                             float64
 15  part_lec                        float64
 16  type_of                         object 
 17  section_lec            

In [115]:
# Объединение колонок part_q и part_lec в part, приведение новой колонки к типу данных int8
merge_dataset['part'] = merge_dataset['part_q'] + merge_dataset['part_lec']
merge_dataset['part'] = merge_dataset['part'].astype('int8')

In [116]:
merge_dataset.drop(['part_q', 'part_lec'], axis=1, inplace=True)
# Удаление дублирующих колонок part_q и part_lec
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 17 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
 9   bundle_id                       float32
 10  correct_answer                  float16
 11  tags                            object 
 12  section_q                       object 
 13  tag                             float64
 14  type_of                         object 
 15  section_lec                     object 
 16  part                            int8   
dtypes: boolean(1), float16(

In [117]:
merge_dataset[['section_q', 'section_lec']] = merge_dataset[['section_q', 'section_lec']].fillna('')
# Заполнение пустых значений пустрой строкой для последующего объединения колонок section_q и section_lec

In [118]:
# Объединение колонок section_q и section_lec в section, приведение новой колонки к типу данных category
merge_dataset['section'] = merge_dataset['section_q'] + merge_dataset['section_lec']
merge_dataset['section'] = merge_dataset['section'].astype('category')

In [119]:
merge_dataset.drop(['section_q', 'section_lec'], axis=1, inplace=True)
# Удаление дублирующих колонок section_q и section_lec
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 16 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   timestamp                       int64   
 1   user_id                         int32   
 2   content_id                      int16   
 3   content_type_id                 int8    
 4   task_container_id               int16   
 5   user_answer                     int8    
 6   answered_correctly              int8    
 7   prior_question_elapsed_time     float32 
 8   prior_question_had_explanation  boolean 
 9   bundle_id                       float32 
 10  correct_answer                  float16 
 11  tags                            object  
 12  tag                             float64 
 13  type_of                         object  
 14  part                            int8    
 15  section                         category
dtypes: boolean(1), category(1), float16(1), float32(2)

In [120]:
# Заполнение пустых значений колонки tag значением -1, приведение к типу данных сначала int, далее str, замена -1 
# на пустую строку для последующего объединения колонок tags и tag
merge_dataset['tag'] = merge_dataset['tag'].fillna(-1).astype('int').astype('str').replace('-1', '')
# Заполнение пустых значений колонки tags пустой строкой, приведение к типу данных str для последующего объединения 
# колонок tags и tag
merge_dataset['tags'] = merge_dataset['tags'].fillna('').astype('str')

In [121]:
# Объединение колонок tags и tag в tags_.
merge_dataset['tags_'] = merge_dataset['tags'] + merge_dataset['tag']

In [122]:
merge_dataset.drop(['tags', 'tag'], axis=1, inplace=True)
# Удаление дублирующих колонок tags и tag
merge_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 15 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   timestamp                       int64   
 1   user_id                         int32   
 2   content_id                      int16   
 3   content_type_id                 int8    
 4   task_container_id               int16   
 5   user_answer                     int8    
 6   answered_correctly              int8    
 7   prior_question_elapsed_time     float32 
 8   prior_question_had_explanation  boolean 
 9   bundle_id                       float32 
 10  correct_answer                  float16 
 11  type_of                         object  
 12  part                            int8    
 13  section                         category
 14  tags_                           object  
dtypes: boolean(1), category(1), float16(1), float32(2), int16(2), int32(1), int64(1), int8(4), objec

In [123]:
# Рассмотрим колонку part - части тестирования от 1 до 7 в вопросах и в лекциях.
# Группировка по вопросам и лекциям, учащимся  в разрезе кол-во частей тестирования.
part_data = merge_dataset.groupby(['content_type_id', 'user_id'])['part'].value_counts()
part_data

content_type_id  user_id     part
0                115         1       37
                             3        3
                             4        3
                             5        2
                             2        1
                                     ..
1                2147469944  5        1
                 2147470770  5        2
                 2147470777  4        3
                             5        3
                 2147482216  2        5
Name: part, Length: 2171755, dtype: int64

In [124]:
part_data = part_data.unstack(level=0) # Перевод вопросы или лекции в колонки.
part_data

content_type_id      0    1
user_id    part            
115        1      37.0  NaN
           2       1.0  NaN
           3       3.0  NaN
           4       3.0  NaN
           5       2.0  NaN
...                ...  ...
2147481750 5      26.0  NaN
2147482216 2     154.0  5.0
           5     121.0  NaN
2147482888 1       1.0  NaN
           5      26.0  NaN

[1826296 rows x 2 columns]

In [125]:
part_data = part_data.unstack() # Перевод частей тестирования в колонки.
part_data

content_type_id     0                                            1           \
part                1      2      3     4      5      6     7    1    2   3   
user_id                                                                       
115              37.0    1.0    3.0   3.0    2.0    NaN   NaN  NaN  NaN NaN   
124               3.0    1.0    3.0   9.0    6.0    4.0   4.0  NaN  NaN NaN   
2746              NaN   17.0    NaN   NaN    2.0    NaN   NaN  NaN  1.0 NaN   
5382             13.0   32.0    NaN   NaN   80.0    NaN   NaN  1.0  NaN NaN   
8623             19.0   44.0    NaN   NaN   31.0    NaN  15.0  2.0  1.0 NaN   
...               ...    ...    ...   ...    ...    ...   ...  ...  ...  ..   
2147470770        3.0   73.0    3.0   9.0  130.0    4.0   4.0  NaN  NaN NaN   
2147470777       80.0   73.0  111.0  81.0  211.0  172.0  24.0  NaN  NaN NaN   
2147481750        NaN   24.0    NaN   NaN   26.0    NaN   NaN  NaN  NaN NaN   
2147482216        NaN  154.0    NaN   NaN  121.0    NaN   NaN  NaN  5.0 NaN   
2147482888        1.0    NaN    NaN   NaN   26.0    NaN   NaN  NaN  NaN NaN   

content_type_id                    
part               4    5   6   7  
user_id                            
115              NaN  NaN NaN NaN  
124              NaN  NaN NaN NaN  
2746             NaN  NaN NaN NaN  
5382             NaN  2.0 NaN NaN  
8623             NaN  NaN NaN NaN  
...              ...  ...  ..  ..  
2147470770       NaN  2.0 NaN NaN  
2147470777       3.0  3.0 NaN NaN  
2147481750       NaN  NaN NaN NaN  
2147482216       NaN  NaN NaN NaN  
2147482888       NaN  NaN NaN NaN  

[393656 rows x 14 columns]

In [126]:
# Объединение part_data с группированной таблицей по вопросам и лекциям, учащимся и доли в % правильных ответов к общему 
# кол-ву вопросов, заданных каждому учащемуся.
merge_part_data = pd.merge(part_data, answered_correct, how='left', on=['user_id'])
merge_part_data

F:\Programs\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,user_id,"(0, 1)","(0, 2)","(0, 3)","(0, 4)","(0, 5)","(0, 6)","(0, 7)","(1, 1)","(1, 2)","(1, 3)","(1, 4)","(1, 5)","(1, 6)","(1, 7)",answered_correctly
0,115,37.0,1.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.57
1,124,3.0,1.0,3.0,9.0,6.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.33
2,2746,NaN,17.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,57.89
3,5382,13.0,32.0,NaN,NaN,80.0,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,67.20
4,8623,19.0,44.0,NaN,NaN,31.0,NaN,15.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,64.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393651,2147470770,3.0,73.0,3.0,9.0,130.0,4.0,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,76.55
393652,2147470777,80.0,73.0,111.0,81.0,211.0,172.0,24.0,NaN,NaN,NaN,3.0,3.0,NaN,NaN,69.15
393653,2147481750,NaN,24.0,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00
393654,2147482216,NaN,154.0,NaN,NaN,121.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,64.36


In [127]:
# Добавление новой колонки cat_score к merge_part_data - категории долей правильных ответов.
merge_part_data['cat_score'] = merge_part_data['answered_correctly'].apply(get_cat_score)
merge_part_data['cat_score']

0         60-80
1         20-40
2         40-60
3         60-80
4         60-80
          ...  
393651    60-80
393652    60-80
393653    60-80
393654    60-80
393655    40-60
Name: cat_score, Length: 393656, dtype: object

In [128]:
# Переименование колонок фрейма merge_part_data
merge_part_data.set_axis\
(['user_id', 'q-1', 'q-2', 'q-3', 'q-4', 'q-5', 'q-6', 'q-7', \
  'lec-1', 'lec-2', 'lec-3', 'lec-4', 'lec-5', 'lec-6', 'lec-7', 'answered_correctly', 'cat_score'],\
 axis='columns', inplace=True)
merge_part_data

,user_id,q-1,q-2,q-3,q-4,q-5,q-6,q-7,lec-1,lec-2,lec-3,lec-4,lec-5,lec-6,lec-7,answered_correctly,cat_score
0,115,37.0,1.0,3.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.57,60-80
1,124,3.0,1.0,3.0,9.0,6.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.33,20-40
2,2746,NaN,17.0,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,57.89,40-60
3,5382,13.0,32.0,NaN,NaN,80.0,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,67.20,60-80
4,8623,19.0,44.0,NaN,NaN,31.0,NaN,15.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,64.22,60-80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393651,2147470770,3.0,73.0,3.0,9.0,130.0,4.0,4.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,76.55,60-80
393652,2147470777,80.0,73.0,111.0,81.0,211.0,172.0,24.0,NaN,NaN,NaN,3.0,3.0,NaN,NaN,69.15,60-80
393653,2147481750,NaN,24.0,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.00,60-80
393654,2147482216,NaN,154.0,NaN,NaN,121.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,64.36,60-80


In [129]:
pd.options.display.max_columns = 60 # Установка отображения 60 колонок

In [130]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и частей тестирования для вопросов и лекций.
# Выведение описательной статистики по частям тестирования для вопросов и лекций в разрезе категорий долей правильных ответов.
part_score_data = merge_part_data.groupby(['cat_score'])\
[['q-1', 'q-2', 'q-3', 'q-4', 'q-5', 'q-6', 'q-7', 'lec-1', 'lec-2', 'lec-3', 'lec-4', 'lec-5', 'lec-6', 'lec-7']]\
.aggregate(['min', 'max', 'median', 'mean'])
part_score_data

q-1                            q-2                             q-3  \
           min     max median       mean  min     max median        mean  min   
cat_score                                                                       
0-20       1.0   198.0    3.0   3.110219  1.0   132.0    1.0    1.825285  3.0   
20-40      1.0   994.0    3.0   5.186912  1.0  2438.0    1.0    5.619204  2.0   
40-60      1.0  2053.0    3.0  17.013858  1.0  4554.0   10.0   29.775061  1.0   
60-80      1.0  4659.0   11.0  43.755722  1.0  6586.0   27.0   94.273606  1.0   
80-100     1.0  3527.0   13.0  56.455264  1.0  5319.0   14.0  105.647047  3.0   

                                      q-4                             q-5  \
              max median        mean  min     max median        mean  min   
cat_score                                                                   
0-20        141.0    3.0    3.038766  3.0    45.0    9.0    8.886492  1.0   
20-40      2632.0    3.0    5.969934  3.0  1895.0    9.0   11.421960  1.0   
40-60      3389.0    3.0   22.387387  1.0  3121.0    9.0   23.296774  1.0   
60-80      4369.0   15.0   76.099722  1.0  3690.0   12.0   69.908848  1.0   
80-100     3267.0   21.0  121.559550  3.0  3973.0   18.0  115.924791  1.0   

                                      q-6                             q-7  \
              max median        mean  min     max median        mean  min   
cat_score                                                                   
0-20        221.0    6.0    7.412478  1.0    48.0    4.0    4.059821  2.0   
20-40      9013.0    6.0   16.264935  2.0  2091.0    4.0    8.888085  1.0   
40-60      8188.0   18.0   64.817522  2.0  6198.0    4.0   28.556289  1.0   
60-80      9584.0   55.0  189.116514  2.0  4204.0   20.0   90.249336  1.0   
80-100     7807.0   21.0  181.045348  2.0  4936.0   32.0  159.446835  2.0   

                                    lec-1                        lec-2        \
              max median       mean   min   max median      mean   min   max   
cat_score                                                                      
0-20        100.0    4.0   4.043439   1.0   8.0    1.0  1.714286   1.0   4.0   
20-40      1240.0    4.0   7.078151   1.0  79.0    2.0  3.405446   1.0  56.0   
40-60      3202.0    4.0  15.920051   1.0  60.0    2.0  3.414004   1.0  57.0   
60-80      3005.0    5.0  44.692414   1.0  61.0    2.0  3.578007   1.0  59.0   
80-100     3135.0   15.0  95.929082   1.0  59.0    2.0  3.615676   1.0  59.0   

                           lec-3                        lec-4               \
          median      mean   min   max median      mean   min   max median   
cat_score                                                                    
0-20         1.0  1.171717   8.0   8.0    8.0  8.000000   1.0   5.0    3.0   
20-40        1.0  1.897923   1.0  19.0    2.0  4.421456   1.0  41.0    2.0   
40-60        2.0  3.536466   1.0  21.0    2.0  3.565375   1.0  37.0    2.0   
60-80        2.0  5.038140   1.0  22.0    2.0  3.567814   1.0  37.0    2.0   
80-100       2.0  4.535954   1.0  20.0    2.0  3.833904   1.0  32.0    2.0   

                    lec-5                          lec-6               \
               mean   min    max median       mean   min   max median   
cat_score                                                               
0-20       3.000000   1.0    2.0    1.0   1.112245   1.0  21.0    1.0   
20-40      6.555328   1.0  141.0    1.0   3.710086   1.0  67.0    2.0   
40-60      3.924615   1.0  158.0    2.0   6.217137   1.0  67.0    2.0   
60-80      3.912391   1.0  142.0    4.0   9.941427   1.0  67.0    3.0   
80-100     4.073480   1.0  140.0    4.0  10.404310   1.0  66.0    3.0   

                    lec-7                         
               mean   min   max median      mean  
cat_score                                         
0-20       2.263158   1.0   1.0    1.0  1.000000  
20-40      7.544889   1.0  27.0    2.0  5.923280  
40-60      5.667844   

1 часть: 
Вопросы. Минимальное значение по всем категориям - 1. Максимальные значения имеют большой разброс от 198 (0-20%) до 4659 (60-80%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения около 60 лекций на человека в категориях от 40 до 100%. 20-40% - максимум - 79. Медиана - 1-2 лекции на человека, средние значения побольше для категорий от 20 до 100% -чуть более 3.
2 часть:
Вопросы. Минимальное значение по всем категориям - 1. Максимальные значения имеют большой разброс от 132 (0-20%) до 6586 (60-80%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения около 60 лекций на человека в категориях от 20 до 100%. Медиана - 1-2 лекции на человека, средние значения побольше для категорий от 40 до 100% - 3-5.
3 часть:
Вопросы. Минимальное значение по категориям - 1-3. Максимальные значения имеют большой разброс от 141 (0-20%) до 4369 (60-80%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1, кроме категории 0-20% (здесь видим, что имеется одно значение 8, которое является максимумом, медианой и средним). Максимальные значения около 20 лекций на человека в категориях от 20 до 100%. Медиана -2 лекции на человека, средние значения для категорий от 20 до 100% - 3-4.
4 часть:
Вопросы. Минимальное значение по всем категориям - 1 или 3. Максимальные значения имеют большой разброс от 45 (0-20%) до 3973 (80-100%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения около 30-40 лекций на человека в категориях от 20 до 100%. 20-40% - максимум - 41. Медиана - 2-3 лекции на человека, средние значения побольше для категорий от 20 до 100% - 4-6 (6,5 для категории 20-40%).
5 часть:
Вопросы. Минимальное значение по всем категориям - 1. Максимальные значения имеют большой разброс от 221 (0-20%) до 9584 (60-80%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения около 140-160 лекций на человека в категориях от 20 до 100%. 40-60% - максимум - 158. Медиана -1-4 лекции на человека, средние значения побольше для категорий от 20 до 100% -3-10, максимально для категории 80-100% - 10,4 лекции на человека.
6 часть:
Вопросы. Минимальное значение по категориям - 1-2. Максимальные значения имеют большой разброс от 48 (0-20%) до 6198 (40-60%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения 66-67 лекций на человека в категориях от 20 до 100%. Медиана - 1-3 лекции на человека, средние значения побольше для категорий от 20 до 100% - 5-7, максимум 7,5 для категории 20-40%.
7 часть:
Вопросы. Минимальное значение по категориям - 1-2. Максимальные значения имеют большой разброс от 100 (0-20%) до 3202 (40-60%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения 26-33 лекций на человека в категориях от 20 до 100%. Медиана - 1-2 лекции на человека, средние значения побольше для категорий от 20 до 100% - 4-6.

In [131]:
# Рассмотрим колонку section - секции listening (part 1-4), reading (part 5-7) в вопросах и в лекциях.
# Группировка по вопросам и лекциям, учащимся  в разрезе кол-во частей тестирования.
section_data = merge_dataset.groupby(['content_type_id', 'user_id'])['section'].value_counts()
section_data

content_type_id  user_id     section  
0                115         listening    44
                             reading       2
                 124         listening    16
                             reading      14
                 2746        listening    17
                                          ..
1                2147469944  reading       1
                 2147470770  reading       2
                 2147470777  listening     3
                             reading       3
                 2147482216  listening     5
Name: section, Length: 985563, dtype: int64

In [132]:
section_data = section_data.unstack(level=0) # Перевод вопросы или лекции в колонки.
section_data

content_type_id           0    1
user_id    section              
115        listening   44.0  NaN
           reading      2.0  NaN
124        listening   16.0  NaN
           reading     14.0  NaN
2746       listening   17.0  1.0
...                     ...  ...
2147481750 reading     26.0  NaN
2147482216 listening  154.0  5.0
           reading    121.0  NaN
2147482888 listening    1.0  NaN
           reading     26.0  NaN

[759338 rows x 2 columns]

In [133]:
section_data = section_data.unstack() # Перевод секций в колонки.
section_data

content_type_id         0                 1        
section         listening reading listening reading
user_id                                            
115                  44.0     2.0       NaN     NaN
124                  16.0    14.0       NaN     NaN
2746                 17.0     2.0       1.0     NaN
5382                 45.0    80.0       1.0     2.0
8623                 63.0    46.0       3.0     NaN
...                   ...     ...       ...     ...
2147470770           88.0   138.0       NaN     2.0
2147470777          345.0   407.0       3.0     3.0
2147481750           24.0    26.0       NaN     NaN
2147482216          154.0   121.0       5.0     NaN
2147482888            1.0    26.0       NaN     NaN

[393656 rows x 4 columns]

In [134]:
# Объединение section_data с группированной таблицей по вопросам и лекциям, учащимся и доли в % правильных ответов к общему 
# кол-ву вопросов, заданных каждому учащемуся.
merge_section_data = pd.merge(section_data, answered_correct, how='left', on=['user_id'])
merge_section_data

F:\Programs\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,user_id,"(0, listening)","(0, reading)","(1, listening)","(1, reading)",answered_correctly
0,115,44.0,2.0,NaN,NaN,69.57
1,124,16.0,14.0,NaN,NaN,23.33
2,2746,17.0,2.0,1.0,NaN,57.89
3,5382,45.0,80.0,1.0,2.0,67.20
4,8623,63.0,46.0,3.0,NaN,64.22
...,...,...,...,...,...,...
393651,2147470770,88.0,138.0,NaN,2.0,76.55
393652,2147470777,345.0,407.0,3.0,3.0,69.15
393653,2147481750,24.0,26.0,NaN,NaN,76.00
393654,2147482216,154.0,121.0,5.0,NaN,64.36


In [135]:
# Добавление новой колонки cat_score к merge_section_data - категории долей правильных ответов.
merge_section_data['cat_score'] = merge_section_data['answered_correctly'].apply(get_cat_score)
merge_section_data['cat_score']

0         60-80
1         20-40
2         40-60
3         60-80
4         60-80
          ...  
393651    60-80
393652    60-80
393653    60-80
393654    60-80
393655    40-60
Name: cat_score, Length: 393656, dtype: object

In [136]:
# Переименование колонок фрейма merge_section_data
merge_section_data.set_axis\
(['user_id', 'q-listening', 'q-reading', 'lec-listening', 'lec-reading', 'answered_correctly', 'cat_score'],\
 axis='columns', inplace=True)
merge_section_data

,user_id,q-listening,q-reading,lec-listening,lec-reading,answered_correctly,cat_score
0,115,44.0,2.0,NaN,NaN,69.57,60-80
1,124,16.0,14.0,NaN,NaN,23.33,20-40
2,2746,17.0,2.0,1.0,NaN,57.89,40-60
3,5382,45.0,80.0,1.0,2.0,67.20,60-80
4,8623,63.0,46.0,3.0,NaN,64.22,60-80
...,...,...,...,...,...,...,...
393651,2147470770,88.0,138.0,NaN,2.0,76.55,60-80
393652,2147470777,345.0,407.0,3.0,3.0,69.15,60-80
393653,2147481750,24.0,26.0,NaN,NaN,76.00,60-80
393654,2147482216,154.0,121.0,5.0,NaN,64.36,60-80


In [137]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и секций тестирования для вопросов и лекций.
# Выведение описательной статистики по секциям тестирования для вопросов и лекций в разрезе категорий долей правильных ответов.
section_score_data = merge_section_data.groupby(['cat_score'])[['q-listening', 'q-reading', 'lec-listening', 'lec-reading']]\
.aggregate(['min', 'max', 'median', 'mean'])
section_score_data

q-listening                             q-reading                  \
                  min      max median        mean       min      max median   
cat_score                                                                     
0-20              1.0    501.0   16.0   14.452278       1.0    255.0   14.0   
20-40             1.0   5958.0   16.0   21.893029       1.0   9308.0   14.0   
40-60             1.0   9612.0   16.0   60.951830       1.0  12946.0   22.0   
60-80             1.0  11045.0   48.0  201.851074       1.0  11365.0   68.0   
80-100            1.0  11778.0   30.0  266.049423       1.0  11721.0   27.0   

                      lec-listening                         lec-reading  \
                 mean           min    max median      mean         min   
cat_score                                                                 
0-20        13.680089           1.0   11.0    1.0  1.455782         1.0   
20-40       26.613425           1.0  124.0    1.0  3.566380         1.0   
40-60       83.839960           1.0  149.0    2.0  5.390938         1.0   
60-80      258.889421           1.0  170.0    3.0  7.863871         1.0   
80-100     312.258934           1.0  160.0    3.0  8.598649         1.0   

                                    
             max median       mean  
cat_score                           
0-20        21.0    1.0   1.310924  
20-40      203.0    1.0   5.441542  
40-60      227.0    2.0   7.533912  
60-80      231.0    5.0  11.916325  
80-100     227.0    6.0  14.094110

Секция listening:
Вопросы. Минимальное значение по всем категориям - 1. Максимальные значения имеют большой разброс от 501 (0-20%) до 11778 (80-100%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения 124-170 лекций на человека в категориях от 20 до 100%, максимум 170 лекций на человека в категории 60-80%. Медиана - 1-3 лекции на человека, средние значения побольше для категорий от 20 до 100% - 3-8, максимальное среднее 8,6 для категории 80-100%.
Секция reading:
Вопросы. Минимальное значение по всем категориям - 1. Максимальные значения имеют большой разброс от 255 (0-20%) до 12946 (40-60%). Медиана и среднее отличаются больше при переходе в категории с большими %. 
Лекции. Минимальное значение по всем категориям - 1. Максимальные значения 200-230 лекций на человека в категориях от 20 до 100%, максимум 231 лекция на человека в категории 60-80%. Медиана - 1-6 лекции на человека, средние значения побольше для категорий от 20 до 100% - 5-14, максимальное среднее 14 для категории 80-100%.

In [138]:
# Рассмотрим колонку type_of - типы тем в лекциях.
# Группировка по лекциям, учащимся  в разрезе типа тем для лекций.
type_data = merge_dataset[merge_dataset['content_type_id'] == 1].groupby(['user_id'])['type_of'].value_counts()
type_data

user_id     type_of         
2746        intention           1
5382        concept             3
8623        concept             3
12741       concept             4
            solving question    2
                               ..
2147469944  concept             3
2147470770  concept             2
2147470777  concept             6
2147482216  intention           4
            concept             1
Name: type_of, Length: 256905, dtype: int64

In [139]:
type_data = type_data.unstack() # Перевод типа тема лекций в колонки.
type_data

type_of,concept,intention,solving question,starter
user_id,,,,
2746,NaN,1.0,NaN,NaN
5382,3.0,NaN,NaN,NaN
8623,3.0,NaN,NaN,NaN
12741,4.0,NaN,2.0,NaN
13134,6.0,1.0,NaN,NaN
...,...,...,...,...
2147419988,27.0,NaN,17.0,NaN
2147469944,3.0,NaN,NaN,NaN
2147470770,2.0,NaN,NaN,NaN


In [140]:
# Объединение type_data с группированной таблицей по вопросам и лекциям, учащимся и доли в % правильных ответов к общему 
# кол-ву вопросов, заданных каждому учащемуся.
merge_type_data = pd.merge(type_data, answered_correct, how='left', on=['user_id'])
merge_type_data

,user_id,concept,intention,solving question,starter,answered_correctly
0,2746,NaN,1.0,NaN,NaN,57.89
1,5382,3.0,NaN,NaN,NaN,67.20
2,8623,3.0,NaN,NaN,NaN,64.22
3,12741,4.0,NaN,2.0,NaN,57.36
4,13134,6.0,1.0,NaN,NaN,70.64
...,...,...,...,...,...,...
149601,2147419988,27.0,NaN,17.0,NaN,50.77
149602,2147469944,3.0,NaN,NaN,NaN,73.26
149603,2147470770,2.0,NaN,NaN,NaN,76.55
149604,2147470777,6.0,NaN,NaN,NaN,69.15


In [141]:
# Добавление новой колонки cat_score к merge_type_data - категории долей правильных ответов.
merge_type_data['cat_score'] = merge_type_data['answered_correctly'].apply(get_cat_score)
merge_type_data['cat_score']

0         40-60
1         60-80
2         60-80
3         40-60
4         60-80
          ...  
149601    40-60
149602    60-80
149603    60-80
149604    60-80
149605    60-80
Name: cat_score, Length: 149606, dtype: object

In [142]:
# Группировка по категориям долей правильных ответов к общему кол-ву вопросов, заданных каждому учащемуся,
# и типа тем лекций.
# Выведение описательной статистики по типам тем лекций в разрезе категорий долей правильных ответов.
type_score_data = merge_type_data.groupby(['cat_score'])[['concept', 'intention', 'solving question', 'starter']]\
.aggregate(['min', 'max', 'median', 'mean'])
type_score_data

concept                          intention                        \
              min    max median       mean       min  max median      mean   
cat_score                                                                    
0-20          1.0   13.0    1.0   1.282511       1.0  2.0    1.0  1.055556   
20-40         1.0  159.0    1.0   4.096183       1.0  7.0    1.0  1.232681   
40-60         1.0  198.0    3.0   7.090486       1.0  7.0    1.0  1.711498   
60-80         1.0  203.0    5.0  12.008049       1.0  7.0    1.0  2.034242   
80-100        1.0  199.0    6.0  13.609542       1.0  8.0    1.0  1.800959   

          solving question                         starter                   
                       min    max median      mean     min  max median mean  
cat_score                                                                    
0-20                   1.0    8.0    1.0  1.483871     NaN  NaN    NaN  NaN  
20-40                  1.0  105.0    2.0  6.119283     NaN  NaN    NaN  NaN  
40-60                  1.0  168.0    2.0  6.210143     1.0  1.0    1.0  1.0  
60-80                  1.0  189.0    3.0  7.901430     1.0  1.0    1.0  1.0  
80-100                 1.0  181.0    4.0  9.951923     NaN  NaN    NaN  NaN

Тип темы лекций concept:
Минимальное значение по всем категориям - 1. Максимальные значения 159-203 лекций на человека в категориях от 20 до 100%, максимум 203 лекции на человека в категории 60-80%. Медиана - 1-6 лекции на человека, средние значения побольше для категорий от 20 до 100% - 4-13, максимальное среднее 13,6 для категории 80-100%.
Тип темы лекций intention:
Минимальное значение по всем категориям - 1. Максимальные значения 2-8 лекций на человека в категориях от 0 до 100%, максимум 8 лекций на человека в категории 80-100%. Медиана - 1 лекция на человека, средние значения побольше для категорий от 40 до 100% - 1-2, максимальное среднее 2 для категории 60-80%.
Тип темы лекций solving question:
Минимальное значение по всем категориям - 1. Максимальные значения 105-189 лекций на человека в категориях от 20 до 100%, максимум 189 лекций на человека в категории 60-80%. Медиана - 1-4 лекции на человека, средние значения побольше для категорий от 20 до 100% - 6-10, максимальное среднее 9,95 для категории 80-100%.
Тип темы лекций starter: 
Минимальное значение по категориям 40-80% - 1. Для остальных параметров значения те же в категориях 40-80%. Лекции данного тип тем проходили 3 человека (см. ниже).

In [143]:
merge_type_data['starter'].value_counts() # 3 учащихся по одному разу проходили тип темы лекции starter

1.0    3
Name: starter, dtype: int64